In [1]:
import tensorflow as tf
import numpy as np
import math
import timeit
import random
import pickle
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
"""
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session
"""

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
    

In [3]:
# Function for permuting and splitting data into training, developement, and test
def import_dataset(address, file_names, train_percent = 80, dev_percent = 10):
    SEED = 455
    random.seed(SEED)
    # Read csv file and create a list of tuples
    images = np.load(address+file_names['images'])
    images = images.astype(float)
    with open(address + file_names['subs'], 'rb') as file_2:
        subs = pickle.load(file_2)
        subs = np.array(subs)
    with open(address + file_names['dict'], 'rb') as file_3:
        dictionary = pickle.load(file_3)
    # Mix data and split into tran, dev, and test sets
    N,W,H,C = np.shape(images)
    indices = np.arange(N)
    random.shuffle(indices)
    images = images[indices]
    subs = subs[indices]
    train_end = int(train_percent*N/100)
    dev_end = train_end + int(dev_percent*N/100)
    X_train = images[:train_end]
    y_train = subs[:train_end]
    X_val = images[train_end:dev_end]
    y_val = subs[train_end:dev_end]
    X_test = images[dev_end:]
    y_test = subs[dev_end:]
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    return X_train, y_train, X_val, y_val, X_test, y_test, dictionary

# Form training, developement, and testing data sets
address = r'data/smallDataset/'
file_names = {}
file_names['images'] = 'data.npy'
file_names['subs'] = 'subredditlabels'
file_names['dict'] = 'subredditIndex'
X_train, y_train, X_val, y_val, X_test, y_test, dictionary = import_dataset(address, file_names)

# Print the sizes as a sanity check
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

('Train data shape: ', (800, 128, 128, 3))
('Train labels shape: ', (800,))
('Validation data shape: ', (100, 128, 128, 3))
('Validation labels shape: ', (100,))
('Test data shape: ', (100, 128, 128, 3))
('Test labels shape: ', (100,))


In [4]:
#sess = get_session()

In [10]:

def resNet_logits(X, trainBool, 
                  NdistinctConvLayers, 
                  Nlayers, Nfilters,
                  kernelSizes):
        
    conv_init = tf.layers.conv2d(
                    inputs = X,
                    filters = 64,
                    kernel_size = [7, 7],
                    padding = "same")
    
    block_input = tf.layers.max_pooling2d(
                    inputs = conv_init,
                    pool_size = [2, 2],
                    strides = 2)
    
    for idl in range(NdistinctConvLayers) :
        if (idl == 0) :
            firstStride = 1
        else :
            firstStride = 2
            
        conv = tf.layers.conv2d(
                        inputs = block_input,
                        filters = Nfilters[idl],
                        kernel_size = kernelSizes[idl],
                        strides = firstStride,
                        padding = "same")
        convBN = tf.layers.batch_normalization(conv, training=trainBool)        
        relu = tf.nn.relu(convBN)
        residual = tf.layers.conv2d(
                        inputs = relu,
                        filters = Nfilters[idl],
                        kernel_size = kernelSizes[idl],
                        strides = 1,
                        padding = "same")
        residualBN = tf.layers.batch_normalization(residual, training=trainBool)
        block_input = tf.nn.relu(residualBN)
        
        for ilr in range(Nlayers[idl] - 1) :
            
            conv = tf.layers.conv2d(
                            inputs = block_input,
                            filters = Nfilters[idl],
                            kernel_size = kernelSizes[idl],
                            strides = 1,
                            padding = "same")
            convBN = tf.layers.batch_normalization(conv, training=trainBool)        
            relu = tf.nn.relu(convBN)
            residual = tf.layers.conv2d(
                            inputs = relu,
                            filters = Nfilters[idl],
                            kernel_size = kernelSizes[idl],
                            strides = 1,
                            padding = "same")
            residualBN = tf.layers.batch_normalization(residual, training=trainBool)        
            output = block_input + residualBN
            outputBN = tf.layers.batch_normalization(output, training=trainBool)
            block_input = tf.nn.relu(outputBN)
            
    convRes = tf.layers.batch_normalization(block_input, training=trainBool)
    
    shape = convRes.get_shape().as_list()
    avgPool = tf.nn.avg_pool(
                value=convRes, 
                ksize=[1,shape[1],shape[2],1],
                strides=[1,1,1,1],
                padding='VALID')
    
    shape = tf.shape(avgPool)
    avgPool_flat = tf.reshape(avgPool,[shape[0], shape[1]*shape[2]*shape[3]])
    logits = tf.layers.dense(inputs=avgPool, units=20)

    return logits

# ResNet loss function
def resNet_loss(logits, y):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    return loss

# Softmax values
def resNet_softmax(logits):
    return tf.nn.softmax(logits)

# ResNet solver
def resNet_solver(learningRate=1e-3, beta1=0.9, beta2=0.999):
    return tf.train.AdamOptimizer(
                learning_rate=learningRate,
                beta1=beta1,
                beta2=beta2)




In [25]:

tf.reset_default_graph()

Nepochs = 10
Niterations = 100
print_every = 50
# batch size
batchSize = 30
# Number of different convolutions
NdistinctConvLayers = 2
# Number of layers for each different convolution
Nlayers = [2, 2]
# Number of filters for each different convolution
Nfilters = [64, 120]
# Size of filters
kernelSizes = [3, 3]

imgHeight = 128
imgWidth = 128

# Batch place holder
batch_X = tf.placeholder(tf.float32, [None, imgHeight, imgWidth, 3])
# Correct classifications
batch_Y = tf.placeholder(tf.int32, [None], 'batch_Y')
# Indicate if training
trainBool = tf.placeholder(tf.bool, name='trainBool')

# Get logits from resNet
with tf.variable_scope("ResNet"):
    logits = resNet_logits(batch_X, trainBool, 
                           NdistinctConvLayers, 
                           Nlayers, Nfilters, kernelSizes)

# Get loss
batch_Y_oneHot = tf.one_hot(indices=batch_Y, depth=20)
loss = resNet_loss(logits, batch_Y_oneHot)

# Get solver
solver = resNet_solver()

# Training vars
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'ResNet')

# Setup for training steps
train_step = solver.minimize(loss, var_list=train_vars)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [26]:
#with tf.Session(graph=graph) as sess:

for ep in range(Nepochs):
    for it in range(Niterations): 
        batchInds = np.random.randint(0,X_train.shape[0], batchSize)
        batchArr_X = X_train[batchInds,:,:,:].astype(np.float32)
        batchArr_Y = y_train[batchInds].astype(np.int32)
        
        feed_dict = {
            batch_X : batchArr_X,
            batch_Y : batchArr_Y,
            trainBool : True
        }
        loss_cur, _ = sess.run([loss, train_step], feed_dict=feed_dict)
        
        if it % print_every == 0:
            print('Epoch: {}      Iter: {}      Loss: {:.4}'.format(ep,it,loss_cur))
        
        
        

(30, 128, 128, 3)
(30,)
float32
int32
Epoch: 0      Iter: 0      Loss: 3.013
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 128, 3)
(30,)
float32
int32
(30, 128, 1

KeyboardInterrupt: 